In [6]:
# Install necessary libraries
!pip install pandas numpy matplotlib seaborn

# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 200)

In [7]:
df = pd.read_csv('/content/Chicago_Crimes_2012_to_2017.csv')


In [8]:
df.shape

(1456714, 23)

In [10]:
df.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,3,10508693,HZ250496,05/03/2016 11:40:00 PM,013XX S SAWYER AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,1022,10.0,24.0,29.0,08B,1154907.0,1893681.0,2016,05/10/2016 03:56:50 PM,41.864073,-87.706819,"(41.864073157, -87.706818608)"
1,89,10508695,HZ250409,05/03/2016 09:40:00 PM,061XX S DREXEL AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,313,3.0,20.0,42.0,08B,1183066.0,1864330.0,2016,05/10/2016 03:56:50 PM,41.782922,-87.604363,"(41.782921527, -87.60436317)"
2,197,10508697,HZ250503,05/03/2016 11:31:00 PM,053XX W CHICAGO AVE,0470,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,False,1524,15.0,37.0,25.0,24,1140789.0,1904819.0,2016,05/10/2016 03:56:50 PM,41.894908,-87.758372,"(41.894908283, -87.758371958)"
3,673,10508698,HZ250424,05/03/2016 10:10:00 PM,049XX W FULTON ST,0460,BATTERY,SIMPLE,SIDEWALK,False,False,1532,15.0,28.0,25.0,08B,1143223.0,1901475.0,2016,05/10/2016 03:56:50 PM,41.885687,-87.749516,"(41.885686845, -87.749515983)"
4,911,10508699,HZ250455,05/03/2016 10:00:00 PM,003XX N LOTUS AVE,0820,THEFT,$500 AND UNDER,RESIDENCE,False,True,1523,15.0,28.0,25.0,06,1139890.0,1901675.0,2016,05/10/2016 03:56:50 PM,41.886297,-87.761751,"(41.886297242, -87.761750709)"


In [11]:
# Check missing coordinates
print("Missing Latitude:", df['Latitude'].isnull().sum())
print("Missing Longitude:", df['Longitude'].isnull().sum())


Missing Latitude: 37083
Missing Longitude: 37083


In [12]:
# Drop rows with missing coordinates
df = df.dropna(subset=['Latitude', 'Longitude'])


In [13]:
# Optionally: Remove coordinates that are clearly invalid (optional)
df = df[(df['Latitude'].between(-90, 90)) & (df['Longitude'].between(-180, 180))]

print("Shape after cleaning coordinates:", df.shape)

Shape after cleaning coordinates: (1419631, 23)


In [14]:
# Parse Date with day-first
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

# Now split into parts
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Date'].dt.hour
df['Weekday'] = df['Date'].dt.day_name()

# Quick check
df[['Date', 'Year', 'Month', 'Day', 'Hour', 'Weekday']].head()


<ipython-input-14-d212a066e6f2>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')


,Date,Year,Month,Day,Hour,Weekday
0,2016-03-05 23:40:00,2016,3,5,23,Saturday
1,2016-03-05 21:40:00,2016,3,5,21,Saturday
2,2016-03-05 23:31:00,2016,3,5,23,Saturday
3,2016-03-05 22:10:00,2016,3,5,22,Saturday
4,2016-03-05 22:00:00,2016,3,5,22,Saturday


In [15]:
df.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Month,Day,Hour,Weekday
0,3,10508693,HZ250496,2016-03-05 23:40:00,013XX S SAWYER AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,1022,10.0,24.0,29.0,08B,1154907.0,1893681.0,2016,05/10/2016 03:56:50 PM,41.864073,-87.706819,"(41.864073157, -87.706818608)",3,5,23,Saturday
1,89,10508695,HZ250409,2016-03-05 21:40:00,061XX S DREXEL AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,313,3.0,20.0,42.0,08B,1183066.0,1864330.0,2016,05/10/2016 03:56:50 PM,41.782922,-87.604363,"(41.782921527, -87.60436317)",3,5,21,Saturday
2,197,10508697,HZ250503,2016-03-05 23:31:00,053XX W CHICAGO AVE,0470,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,False,1524,15.0,37.0,25.0,24,1140789.0,1904819.0,2016,05/10/2016 03:56:50 PM,41.894908,-87.758372,"(41.894908283, -87.758371958)",3,5,23,Saturday
3,673,10508698,HZ250424,2016-03-05 22:10:00,049XX W FULTON ST,0460,BATTERY,SIMPLE,SIDEWALK,False,False,1532,15.0,28.0,25.0,08B,1143223.0,1901475.0,2016,05/10/2016 03:56:50 PM,41.885687,-87.749516,"(41.885686845, -87.749515983)",3,5,22,Saturday
4,911,10508699,HZ250455,2016-03-05 22:00:00,003XX N LOTUS AVE,0820,THEFT,$500 AND UNDER,RESIDENCE,False,True,1523,15.0,28.0,25.0,06,1139890.0,1901675.0,2016,05/10/2016 03:56:50 PM,41.886297,-87.761751,"(41.886297242, -87.761750709)",3,5,22,Saturday


In [16]:

df[['Date', 'Year', 'Month', 'Day', 'Hour', 'Weekday']].head()

,Date,Year,Month,Day,Hour,Weekday
0,2016-03-05 23:40:00,2016,3,5,23,Saturday
1,2016-03-05 21:40:00,2016,3,5,21,Saturday
2,2016-03-05 23:31:00,2016,3,5,23,Saturday
3,2016-03-05 22:10:00,2016,3,5,22,Saturday
4,2016-03-05 22:00:00,2016,3,5,22,Saturday


In [17]:
# Remove exact duplicate rows
df = df.drop_duplicates()
print("Shape after dropping duplicates:", df.shape)


Shape after dropping duplicates: (1419631, 27)


In [19]:
df.describe()

,Unnamed: 0,ID,Date,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Year,Latitude,Longitude,Month,Day,Hour
count,1.419631e+06,1.419631e+06,1419631,1.419631e+06,1.419630e+06,1.419617e+06,1.419606e+06,1.419631e+06,1.419631e+06,1.419631e+06,1.419631e+06,1.419631e+06,1.419631e+06,1.419631e+06,1.419631e+06
mean,3.273571e+06,9.574934e+06,2014-05-08 04:25:26.527790848,1.150502e+03,1.125737e+01,2.286073e+01,3.746315e+01,1.164398e+06,1.885523e+06,2.013851e+03,4.184147e+01,-8.767224e+01,6.488135e+00,1.574764e+01,1.323373e+01
min,3.000000e+00,2.022400e+04,2012-01-01 00:00:00,1.110000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.012000e+03,3.661945e+01,-9.168657e+01,1.000000e+00,1.000000e+00,0.000000e+00
25%,2.696712e+06,8.987078e+06,2013-01-23 17:29:00,6.130000e+02,6.000000e+00,1.000000e+01,2.300000e+01,1.152544e+06,1.858762e+06,2.013000e+03,4.176787e+01,-8.771528e+01,4.000000e+00,8.000000e+00,9.000000e+00
50%,3.052442e+06,9.576348e+06,2014-04-09 06:30:00,1.024000e+03,1.000000e+01,2.300000e+01,3.200000e+01,1.166021e+06,1.891502e+06,2.014000e+03,4.185797e+01,-8.766613e+01,7.000000e+00,1.600000e+01,1.400000e+01
75%,3.409860e+06,1.019197e+07,2015-08-07 19:00:00,1.711000e+03,1.700000e+01,3.400000e+01,5.700000e+01,1.176363e+06,1.908713e+06,2.015000e+03,4.190529e+01,-8.762813e+01,9.000000e+00,2.300000e+01,1.900000e+01
max,6.253474e+06,1.082334e+07,2017-12-01 21:31:00,2.535000e+03,3.100000e+01,5.000000e+01,7.700000e+01,1.205119e+06,1.951573e+06,2.017000e+03,4.202271e+01,-8.752453e+01,1.200000e+01,3.100000e+01,2.300000e+01
std,1.183632e+06,8.011899e+05,NaN,6.916767e+02,6.904887e+00,1.379728e+01,2.143258e+01,1.850835e+04,3.424775e+04,1.423583e+00,9.430126e-02,6.661726e-02,3.389803e+00,8.735946e+00,6.593778e+00


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1419631 entries, 0 to 1456713
Data columns (total 27 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   Unnamed: 0            1419631 non-null  int64         
 1   ID                    1419631 non-null  int64         
 2   Case Number           1419631 non-null  object        
 3   Date                  1419631 non-null  datetime64[ns]
 4   Block                 1419631 non-null  object        
 5   IUCR                  1419631 non-null  object        
 6   Primary Type          1419631 non-null  object        
 7   Description           1419631 non-null  object        
 8   Location Description  1418405 non-null  object        
 9   Arrest                1419631 non-null  bool          
 10  Domestic              1419631 non-null  bool          
 11  Beat                  1419631 non-null  int64         
 12  District              1419630 non-null  float64

In [21]:
# Standardize 'Primary Type'
df['Primary Type'] = df['Primary Type'].str.strip().str.title()

# Standardize 'Location Description'
# First fill missing with 'Unknown' (because 1418405 non-null out of 1419631)
df['Location Description'] = df['Location Description'].fillna('Unknown')
df['Location Description'] = df['Location Description'].str.strip().str.title()

# Quick check
df[['Primary Type', 'Location Description']].head(10)


,Primary Type,Location Description
0,Battery,Apartment
1,Battery,Residence
2,Public Peace Violation,Street
3,Battery,Sidewalk
4,Theft,Residence
5,Battery,Street
6,Battery,Cha Hallway/Stairwell/Elevator
7,Battery,Residence Porch/Hallway
8,Battery,Sidewalk
9,Battery,Street


In [22]:
# Before Cleaning - Sample view
print("Before Cleaning:")
print(df[['Primary Type', 'Location Description']].sample(5))

# Standardize
df['Primary Type'] = df['Primary Type'].str.strip().str.title()
df['Location Description'] = df['Location Description'].fillna('Unknown')
df['Location Description'] = df['Location Description'].str.strip().str.title()

# After Cleaning - Sample view
print("\nAfter Cleaning:")
print(df[['Primary Type', 'Location Description']].sample(5))


Before Cleaning:
            Primary Type  Location Description
946818             Theft             Cta Train
514744           Assault              Sidewalk
1171173  Criminal Damage                Street
1169780    Other Offense  Sports Arena/Stadium
469397           Assault             Apartment

After Cleaning:
              Primary Type    Location Description
155184            Burglary               Residence
946198   Weapons Violation               Apartment
238332           Narcotics  Vehicle Non-Commercial
112018               Theft                  Street
1106947    Criminal Damage               Apartment


In [23]:
# Check for full duplicates
print("Full duplicate rows before:", df.duplicated().sum())




Full duplicate rows before: 0


In [24]:
df.shape

(1419631, 27)

In [25]:
# Check for full duplicates
print("Full duplicate rows before:", df.duplicated().sum())

# Remove full duplicate rows
df = df.drop_duplicates()

# Confirm
print("Full duplicate rows after:", df.duplicated().sum())
print("New shape after dropping duplicates:", df.shape)


Full duplicate rows before: 0
Full duplicate rows after: 0
New shape after dropping duplicates: (1419631, 27)


In [26]:
# Check if 'ID' has duplicates
print("Duplicate IDs:", df['ID'].duplicated().sum())

# Check if 'Case Number' has duplicates
print("Duplicate Case Numbers:", df['Case Number'].duplicated().sum())


Duplicate IDs: 0
Duplicate Case Numbers: 107


In [27]:
# Group by Community Area and count incidents
spatial_density = df.groupby('Community Area').size().reset_index(name='Incidents_Per_Community')

# Merge this density back into main DataFrame
df = df.merge(spatial_density, on='Community Area', how='left')

# Quick check
df[['Community Area', 'Incidents_Per_Community']].head(10)


,Community Area,Incidents_Per_Community
0,29.0,44822.0
1,42.0,21293.0
2,25.0,92713.0
3,25.0,92713.0
4,25.0,92713.0
5,44.0,32266.0
6,35.0,12979.0
7,38.0,17665.0
8,1.0,19947.0
9,67.0,40162.0


In [28]:
# Step 1: Make sure 'Date' is the index
df = df.set_index('Date')

# Step 2: Create a daily incidents DataFrame
daily_incidents = df.resample('D').size()

# Step 3: Calculate the 7-day rolling average
rolling_7day_avg = daily_incidents.rolling(window=7, min_periods=1).mean()

# Step 4: Merge the rolling average back into the original DataFrame
# First reset index because our main df needs Date as a column
df = df.reset_index()

# Merge the rolling 7-day average into the main DataFrame
# We map the rolling average value for each date
df['Rolling_7day_Avg'] = df['Date'].map(rolling_7day_avg)

# Quick check
df[['Date', 'Rolling_7day_Avg']].head(10)


,Date,Rolling_7day_Avg
0,2016-03-05 23:40:00,NaN
1,2016-03-05 21:40:00,NaN
2,2016-03-05 23:31:00,NaN
3,2016-03-05 22:10:00,NaN
4,2016-03-05 22:00:00,NaN
5,2016-03-05 22:35:00,NaN
6,2016-03-05 22:30:00,NaN
7,2016-03-05 21:30:00,NaN
8,2016-03-05 16:00:00,NaN
9,2016-03-05 22:30:00,NaN


In [29]:
# Extract just the date part (no time)
df['Date_only'] = df['Date'].dt.date


In [30]:
# Group incidents by Date_only
daily_incidents = df.groupby('Date_only').size()


In [31]:
# Calculate rolling average
rolling_7day_avg = daily_incidents.rolling(window=7, min_periods=1).mean()


In [32]:
# Map rolling average back to the original dataframe
df['Rolling_7day_Avg'] = df['Date_only'].map(rolling_7day_avg)


In [33]:
# Quick check
df[['Date', 'Date_only', 'Rolling_7day_Avg']].head(10)


,Date,Date_only,Rolling_7day_Avg
0,2016-03-05 23:40:00,2016-03-05,604.428571
1,2016-03-05 21:40:00,2016-03-05,604.428571
2,2016-03-05 23:31:00,2016-03-05,604.428571
3,2016-03-05 22:10:00,2016-03-05,604.428571
4,2016-03-05 22:00:00,2016-03-05,604.428571
5,2016-03-05 22:35:00,2016-03-05,604.428571
6,2016-03-05 22:30:00,2016-03-05,604.428571
7,2016-03-05 21:30:00,2016-03-05,604.428571
8,2016-03-05 16:00:00,2016-03-05,604.428571
9,2016-03-05 22:30:00,2016-03-05,604.428571


In [34]:
# prompt: continue with feature 3:
# Weekend/weekday crime flags

# Create 'Is_Weekend' flag
df['Is_Weekend'] = df['Weekday'].isin(['Saturday', 'Sunday'])

# Create 'Is_Weekday' flag (opposite of 'Is_Weekend')
df['Is_Weekday'] = ~df['Is_Weekend']

# Quick check
df[['Weekday', 'Is_Weekend', 'Is_Weekday']].head(10)


,Weekday,Is_Weekend,Is_Weekday
0,Saturday,True,False
1,Saturday,True,False
2,Saturday,True,False
3,Saturday,True,False
4,Saturday,True,False
5,Saturday,True,False
6,Saturday,True,False
7,Saturday,True,False
8,Saturday,True,False
9,Saturday,True,False


In [35]:
# prompt: continue with feature 4:
# Repeat incident probability

# Calculate the probability of an incident happening on each day of the week.
weekday_probabilities = df.groupby('Weekday')['ID'].count() / len(df)

# Print the probabilities.
print(weekday_probabilities)

# Function to predict the probability of an incident based on the day of the week
def predict_incident_probability(day_of_week):
    """Predicts the probability of an incident based on the day of the week.

    Args:
        day_of_week: The day of the week (e.g., 'Monday', 'Tuesday').

    Returns:
        The probability of an incident on that day of the week, or None if the day is invalid.
    """
    day_of_week = day_of_week.title()  # Standardize input
    if day_of_week in weekday_probabilities:
        return weekday_probabilities[day_of_week]
    else:
        return None

# Example usage:
probability_on_monday = predict_incident_probability('monday')
print(f"Probability of an incident on Monday: {probability_on_monday}")


Weekday
Friday       0.147738
Monday       0.141546
Saturday     0.143207
Sunday       0.140699
Thursday     0.142159
Tuesday      0.141771
Wednesday    0.142880
Name: ID, dtype: float64
Probability of an incident on Monday: 0.14154593693713366


In [36]:
# prompt: Reshape step by step,
#  data for analysis:
# ■ Unpivot crime counts
# ■ Pivot monthly summaries for time-series analysis

import pandas as pd
# Unpivot crime counts (assuming you want to unpivot 'Primary Type' counts)
# First, group by date and crime type, then count occurrences
crime_counts = df.groupby(['Date', 'Primary Type']).size().reset_index(name='Counts')

# Now, pivot the table to show monthly crime summaries
monthly_crimes = crime_counts.groupby(['Date', 'Primary Type'])['Counts'].sum().reset_index()

# Convert 'Date' to datetime if it isn't already
monthly_crimes['Date'] = pd.to_datetime(monthly_crimes['Date'])


# Extract year and month
monthly_crimes['Year'] = monthly_crimes['Date'].dt.year
monthly_crimes['Month'] = monthly_crimes['Date'].dt.month

# Group by year, month, and primary type and sum the counts
monthly_summaries = monthly_crimes.groupby(['Year', 'Month', 'Primary Type'])['Counts'].sum().reset_index()

# Pivot the monthly summaries for time-series analysis
monthly_pivot = monthly_summaries.pivot_table(index=['Year', 'Month'], columns='Primary Type', values='Counts', fill_value=0)

# Display the pivoted data
print(monthly_pivot.head())


Primary Type  Arson  Assault  Battery  Burglary  \
Year Month                                        
2012 1         39.0   1451.0   4769.0    1817.0   
     2         29.0   1495.0   4360.0    1556.0   
     3         42.0   1841.0   5278.0    1756.0   
     4         42.0   1646.0   4964.0    1719.0   
     5         41.0   1786.0   5587.0    1872.0   

Primary Type  Concealed Carry License Violation  Crim Sexual Assault  \
Year Month                                                             
2012 1                                      0.0                154.0   
     2                                      0.0                105.0   
     3                                      0.0                107.0   
     4                                      0.0                 90.0   
     5                                      0.0                119.0   

Primary Type  Criminal Damage  Criminal Trespass  Deceptive Practice  \
Year Month                                                       

In [37]:
print(monthly_pivot.index.get_level_values('Year').unique())


Index([2012, 2013, 2014, 2015, 2016, 2017], dtype='int32', name='Year')


In [38]:
monthly_2013 = monthly_pivot.loc[2013]
print(monthly_2013)


Primary Type  Arson  Assault  Battery  Burglary  \
Month                                             
1              28.0   1541.0   4425.0    1555.0   
2              22.0   1251.0   3838.0    1203.0   
3              30.0   1448.0   4393.0    1321.0   
4              33.0   1609.0   4664.0    1421.0   
5              44.0   1754.0   5013.0    1552.0   
6              30.0   1656.0   5135.0    1456.0   
7              39.0   1647.0   5016.0    1525.0   
8              37.0   1568.0   4792.0    1703.0   
9              26.0   1524.0   4400.0    1559.0   
10             30.0   1436.0   4409.0    1599.0   
11             19.0   1312.0   3941.0    1501.0   
12             26.0   1179.0   3844.0    1430.0   

Primary Type  Concealed Carry License Violation  Crim Sexual Assault  \
Month                                                                  
1                                           0.0                123.0   
2                                           0.0                 81.0 

In [39]:
monthly_2012_2013 = monthly_pivot.loc[[2012, 2013]]
print(monthly_2012_2013)


Primary Type  Arson  Assault  Battery  Burglary  \
Year Month                                        
2012 1         39.0   1451.0   4769.0    1817.0   
     2         29.0   1495.0   4360.0    1556.0   
     3         42.0   1841.0   5278.0    1756.0   
     4         42.0   1646.0   4964.0    1719.0   
     5         41.0   1786.0   5587.0    1872.0   
     6         46.0   1759.0   5242.0    2017.0   
     7         45.0   1796.0   5527.0    2143.0   
     8         46.0   1836.0   5190.0    2090.0   
     9         51.0   1641.0   4701.0    1988.0   
     10        30.0   1659.0   4807.0    2057.0   
     11        24.0   1525.0   4455.0    1997.0   
     12        34.0   1459.0   4240.0    1825.0   
2013 1         28.0   1541.0   4425.0    1555.0   
     2         22.0   1251.0   3838.0    1203.0   
     3         30.0   1448.0   4393.0    1321.0   
     4         33.0   1609.0   4664.0    1421.0   
     5         44.0   1754.0   5013.0    1552.0   
     6         30.0   1656.0   

In [43]:
jan_2012 = monthly_pivot.loc[(2012, 12)]
print(jan_2012)


Primary Type
Arson                                  34.0
Assault                              1459.0
Battery                              4240.0
Burglary                             1825.0
Concealed Carry License Violation       0.0
Crim Sexual Assault                   101.0
Criminal Damage                      2741.0
Criminal Trespass                     641.0
Deceptive Practice                   1040.0
Gambling                               40.0
Homicide                               31.0
Human Trafficking                       0.0
Interference With Public Officer       95.0
Intimidation                            9.0
Kidnapping                             17.0
Liquor Law Violation                   33.0
Motor Vehicle Theft                  1420.0
Narcotics                            2570.0
Non - Criminal                          0.0
Non-Criminal                            0.0
Non-Criminal (Subject Specified)        0.0
Obscenity                               3.0
Offense Involving C

In [44]:
monthly_pivot_reset = monthly_pivot.reset_index()
print(monthly_pivot_reset.head())


Primary Type  Year  Month  Arson  Assault  Battery  Burglary  \
0             2012      1   39.0   1451.0   4769.0    1817.0   
1             2012      2   29.0   1495.0   4360.0    1556.0   
2             2012      3   42.0   1841.0   5278.0    1756.0   
3             2012      4   42.0   1646.0   4964.0    1719.0   
4             2012      5   41.0   1786.0   5587.0    1872.0   

Primary Type  Concealed Carry License Violation  Crim Sexual Assault  \
0                                           0.0                154.0   
1                                           0.0                105.0   
2                                           0.0                107.0   
3                                           0.0                 90.0   
4                                           0.0                119.0   

Primary Type  Criminal Damage  Criminal Trespass  Deceptive Practice  \
0                      2824.0              678.0              1227.0   
1                      2539.0         

In [45]:
import sqlite3


In [48]:
!pip install psycopg2-binary


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.7 MB/s eta 0:00:00


In [49]:
import sqlite3

# Connect to a local SQLite database
db_path = 'crime_data.db'  # You can make this configurable
conn = sqlite3.connect(db_path)


In [51]:
# Get unique crime types
crime_types = df[['Primary Type']].drop_duplicates().reset_index(drop=True)

# Add a Primary_Type_ID
crime_types['Primary_Type_ID'] = crime_types.index + 1

# Reorder columns
crime_types = crime_types[['Primary_Type_ID', 'Primary Type']]

crime_types.head()


,Primary_Type_ID,Primary Type
0,1,Battery
1,2,Public Peace Violation
2,3,Theft
3,4,Weapons Violation
4,5,Robbery


In [52]:
# Get unique location descriptions
locations = df[['Location Description']].drop_duplicates().reset_index(drop=True)

# Add a Location_ID
locations['Location_ID'] = locations.index + 1

# Reorder columns
locations = locations[['Location_ID', 'Location Description']]

locations.head()


,Location_ID,Location Description
0,1,Apartment
1,2,Residence
2,3,Street
3,4,Sidewalk
4,5,Cha Hallway/Stairwell/Elevator


In [53]:
# Merge with Crime_Types to get Primary_Type_ID
df = df.merge(crime_types, on='Primary Type', how='left')

# Merge with Locations to get Location_ID
df = df.merge(locations, on='Location Description', how='left')

# Create Incident_ID
df['Incident_ID'] = df.index + 1

# Select only relevant columns
incidents = df[['Incident_ID', 'Date', 'Primary_Type_ID', 'Location_ID']]

incidents.head()


,Incident_ID,Date,Primary_Type_ID,Location_ID
0,1,2016-03-05 23:40:00,1,1
1,2,2016-03-05 21:40:00,1,2
2,3,2016-03-05 23:31:00,2,3
3,4,2016-03-05 22:10:00,1,4
4,5,2016-03-05 22:00:00,3,2


In [54]:
# Save tables
crime_types.to_sql('Crime_Types', conn, index=False, if_exists='replace')
locations.to_sql('Locations', conn, index=False, if_exists='replace')
incidents.to_sql('Incidents', conn, index=False, if_exists='replace')


1419631

In [55]:
# Create a cursor
cursor = conn.cursor()

# Example: View first 5 incidents
cursor.execute('SELECT * FROM Incidents LIMIT 5;')
for row in cursor.fetchall():
    print(row)


(1, '2016-03-05 23:40:00', 1, 1)
(2, '2016-03-05 21:40:00', 1, 2)
(3, '2016-03-05 23:31:00', 2, 3)
(4, '2016-03-05 22:10:00', 1, 4)
(5, '2016-03-05 22:00:00', 3, 2)


In [60]:
query = """
SELECT i.Incident_ID, i.Date, ct."Primary Type", l."Location Description"
FROM Incidents i
JOIN Crime_Types ct ON i.Primary_Type_ID = ct.Primary_Type_ID
JOIN Locations l ON i.Location_ID = l.Location_ID
LIMIT 10;
"""

for row in cursor.execute(query):
    print(row)


(1, '2016-03-05 23:40:00', 'Battery', 'Apartment')
(2, '2016-03-05 21:40:00', 'Battery', 'Residence')
(3, '2016-03-05 23:31:00', 'Public Peace Violation', 'Street')
(4, '2016-03-05 22:10:00', 'Battery', 'Sidewalk')
(5, '2016-03-05 22:00:00', 'Theft', 'Residence')
(6, '2016-03-05 22:35:00', 'Battery', 'Street')
(7, '2016-03-05 22:30:00', 'Battery', 'Cha Hallway/Stairwell/Elevator')
(8, '2016-03-05 21:30:00', 'Battery', 'Residence Porch/Hallway')
(9, '2016-03-05 16:00:00', 'Battery', 'Sidewalk')
(10, '2016-03-05 22:30:00', 'Battery', 'Street')
